# Discrete sampling
In the last problem of the first session we sampled with given probabilities using prefix sums and binary search. The sampling code is the bottleneck of the whole process, its running times is 3 times higher than the code for uniform probabilities, and the difference would only increase for larger number of outcomes. In the next two problems we discuss two simple, but powerful techniques one can use to sample in time $O(1)$.

**Problem 2a.** Consider the problem of sampling with known probabilities $p_1,\ldots,p_d$. Suppose that you have a black-box that samples with probabilities $q_1,\ldots,q_d$ that are close to $p_1,\ldots,p_d$, say
$$ \forall_{i=1,\ldots,n} p_i \le (1+\varepsilon)q_i.$$

* How can you use this black-box to sample with probabilities $p_1,\ldots,p_d$? It is expected, that the running time of the algorithm would be non-deterministic.
* Prove that your algorithm is correct.
* Implement the algorithm and use it to give a faster implementation for **Problem 1c**.

**Problem 2b.** One of the reasons this implementation is not significantly faster than the one in **Problem 1c** , apart from $d$ being rather small, is that we are using Python's interpreter a bit too much, and Python is slow. One way around this is usually to use a library function - **searchsorted** is much faster than an equivalent code implemented in pure Python. But even if the functionality you need is not implemented in a lower level language as
a library function, there is still hope. You can try to implement it using optimized array algebra, for example using **numpy**. In this problem, your task is to rewrite the previous algorithm, so that the amount of *looping* is reduced to a minimum necessary. In particular, you should create a *vectorized* version of random dates generation (in bulk), while the actual search for duplicates remains a loop with a **set**. Here are some useful tips:
   * You can perform arithmetic, comparisons, etc. on **numpy** arrays.
   * You can generate whole **numpy** arrays of random numbers at once.
   * You can even perform parallel look-up like in the example below.

In [1]:
X = np.array([10,3,7])
I = np.array([1,1,2,2])
print(X[I])
X = np.array([[1,2],[3,4]])
I = np.array([0,0,1])
J = np.array([1,0,1])
print(X[I,J])

NameError: name 'np' is not defined

**Problem 2c (Squaring the histogram).** In this problem, we again want to sample with known probabilities $p_1,\ldots,p_n$, but this time we make no assumptions on $p_i$. Consider the following algorithm:
   * Let $V$ be the mean of $p_i$, i.e. $V=\frac{1}{n}$.
   * Create $n$ buckets, each with volume $V$, put each $p_i$ into a separate bucket.
   * Until there exists a bucket $A$ that is not full, find a bucket $B$ that overflows, and trasfer probability from $B$ to $A$ until $A$ is exactly full

Show that:
   * This algorithm always ends.
   * When it ends, each bucket contains pieces of exactly two $p_i$'s.

How to use the result of this algorithm to sample with probabilities $p_i$. Argue that your algorithm is correct and implement it. The sampling part should be *vectorized*. Use this algorithm to sample birthdates again, and test its efficiency.

Operation of transferring probability from one bucket to another doesn't change mean ($V = 1/n$).

In each step we have two options:

In [4]:
import numpy as np

p = np.linspace(0, 1, num=5)

def squaring_histogram(p):
    B = np.array(p, dtype='f') # Bucket
    N = B.size

    mean = np.mean(B)
    
    Donor = np.arange(N)
    Probs = np.empty(N)
    Probs.fill(mean)

    while np.count_nonzero(B < mean):

        i = np.where(B < mean)[0][0]
        j = np.where(B > mean)[0][0]

        Donor[i] = j
        Probs[i] = B[i]
        
        B[j] = B[j] - (mean - B[i])
        B[i] = mean
        
    return Donor, Probs

print squaring_histogram(p)

(array([3, 4, 2, 4, 4]), array([ 0.  ,  0.25,  0.5 ,  0.25,  0.5 ]))


In [12]:
import pandas as pd

data = pd.read_csv('us_births_69_88.csv')
births = np.array(data['births'])

n = len(births)
mean = np.mean(births)
K = 400

sample_days = np.random.randint(0, n-1, K)
sample_births = np.random.random_sample(K) * mean

Donor, Probs = squaring_histogram(births)

# Brutal for
days = []
for i in range(400):
    day = sample_days[i]
    day_prob = sample_births[i]
    
    if day_prob < Probs[day]:
        days.append(day)
    else:
        days.append(Donor[day])

print days

sample_days[np.where(sample_days < Probs[sample_days])]

[63, 293, 114, 368, 35, 59, 114, 357, 276, 230, 163, 2, 170, 158, 201, 255, 325, 163, 286, 332, 348, 93, 228, 131, 183, 240, 49, 262, 370, 51, 67, 103, 150, 173, 364, 261, 159, 184, 220, 317, 275, 127, 166, 237, 346, 219, 287, 272, 360, 244, 230, 196, 248, 201, 241, 50, 10, 122, 369, 260, 48, 200, 229, 299, 329, 151, 276, 189, 93, 299, 260, 184, 230, 215, 15, 147, 158, 268, 143, 16, 296, 291, 89, 206, 362, 230, 12, 95, 238, 95, 81, 206, 341, 90, 33, 257, 111, 299, 174, 176, 45, 215, 281, 355, 97, 250, 217, 158, 77, 288, 318, 257, 67, 100, 272, 178, 197, 271, 214, 37, 17, 140, 213, 183, 298, 362, 104, 291, 195, 141, 42, 256, 210, 322, 190, 70, 84, 346, 30, 110, 131, 103, 161, 93, 6, 232, 155, 332, 368, 17, 128, 150, 282, 8, 56, 56, 85, 215, 35, 316, 72, 104, 367, 191, 130, 82, 238, 167, 203, 322, 63, 177, 206, 28, 93, 150, 265, 233, 62, 13, 53, 235, 182, 70, 26, 89, 237, 244, 284, 246, 212, 222, 205, 115, 220, 95, 348, 275, 249, 32, 209, 27, 152, 219, 129, 286, 50, 159, 40, 145, 107, 36

array([ 63, 293, 114, 368,  35,  59, 114, 357, 276, 225,  61,   2, 170,
        59, 201, 250, 325,  61, 285, 332, 348,  93, 223, 131, 183, 235,
        49, 262, 370,  51,  67, 103, 150, 173, 364, 261, 159, 103, 215,
       317, 275, 127, 166, 237, 346, 214, 286, 271, 360, 239, 225, 196,
       243, 158, 241,  50,  10, 122, 369, 260,  48, 200, 229, 298, 329,
       151, 276, 189,  93, 298, 256, 184, 225, 208,  15, 147,  59, 268,
       143,  16, 295, 290,  89, 206, 362, 225,  12,  95, 233,  95,  81,
       206, 341,  90,  33, 257, 111, 298, 174, 176,  45, 208, 281, 355,
        97, 250, 217,  59,  77, 287, 318, 253,  67, 100, 271, 178, 197,
       270, 203,  37,  17, 140, 202, 183, 298, 362, 104, 290, 195, 141,
        42, 256, 193, 322, 190,  70,  84, 346,  30, 110, 131, 103, 161,
        93,   6, 227, 155, 332, 368,  17, 128, 150, 282,   8,  56,  56,
        85, 208,  35, 316,  72, 104, 367, 191, 130,  82, 233, 167, 163,
       322,  63, 177, 206,  28,  93, 150, 261, 228,  62,  13,  5

**Problem 2d.** Show that the frequency histogram for empirical birthday frequencies can actually be computed exactly, and implement your idea. To this end, design a recurence relation using conditional probabilities and use dynamic programming.